In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings 
warnings.filterwarnings('ignore') 


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [62]:
data = pd.read_excel("/content/drive/MyDrive/LCF_Internal/Renewable_Model/EDA Approach-1/XGBoost Implementation/logistic_v9/v9_data.xlsx", sheet_name = 'v9_test')
data.head()

,OpportunityID,CreatedDate,FundedDate,OpportunityName,Priordeal_OppID,Renewal_PreviousCollectionStatus,Renewal_PreviousPace,Renewal_PreviousPaidPercentageOfRTR,Renewal_PreviousCreditScore,Renewal_CountPreviousDeals,...,Industry,Industry_Group__c,Avg_ReturnItemdays,LatestMonth_ReturnItemDays,PctChangeInReveune,PctAvgDailybalance2AvgMonthlyDeposits,TwoMonthAvg_AdjustedMonthlyDeposits,AdjustedMonthlyDeposits,drop_percent,Drop_category
0,0062M00000nqf6oQAA,2021-12-20 21:03:51,2022-01-06,G Smith Handyman Group-Renewal Deal-12202021,0062M00000nYhn5QAC,Other,100.00,0.90,642.0,1,...,Services,76 - Miscellaneous Repair & Installation Services,0.25,NaN,-0.272301,0.219321,10817.03,7834.00,-27.577163,Negative
1,0062M00000nqfF3QAI,2021-12-20 21:45:40,2022-01-05,Green Collar Cleaners-Renewal Deal-12202021,0062M00000nZHYfQAO,Other,100.00,0.78,507.0,1,...,Services,72 - Personal Services,1.00,NaN,0.125156,0.049968,16132.02,16735.22,3.739147,Positive
2,0062M00000nqjtvQAA,2021-12-21 19:11:21,2022-01-18,JC Building And Remodeling-Renewal Deal-12212021,0062M00000na4fuQAA,Other,100.00,0.49,611.0,3,...,Construction,15 - Building Construction General Contractors...,0.25,0.0,0.189767,0.229279,41386.19,34545.34,-16.529306,Negative
3,0062M00000nqs8FQAQ,2021-12-23 15:58:53,2022-01-13,Sykes Funeral Trade Service-Renewal Deal-12232021,0062M00000nYYdkQAG,Other,100.00,0.60,762.0,1,...,Services,72 - Personal Services,0.00,NaN,-0.047224,1.201039,7885.00,2220.00,-71.845275,Negative
4,0062M00000nr94cQAA,2021-12-28 21:45:27,2022-01-04,Monrovia Lounge-Renewal Deal-12282021,0062M00000nWwtLQAS,Other,98.33,0.56,614.0,1,...,Services,90 - Restaurants & Bars,0.00,0.0,-0.082257,0.193480,19300.74,15851.61,-17.870454,Negative


In [63]:
data.shape

(488, 49)

In [64]:
data.isna().sum().sort_values(ascending = False)/len(data)*100

HBwoLCF                                  97.336066
NetWO                                    76.434426
Renewal_PreviousHBwoLCF                  65.983607
LatestMonth_ReturnItemDays               34.016393
Renewal_PreviousCreditScore               2.254098
PctChangeInReveune                        2.254098
Credit_Score__c                           1.844262
Renewal_PreviousAvgDailyBalance           1.639344
Renewal_PreviousAvgMonthRevenue           1.639344
Renewal_PreviousPace                      0.409836
Renewal_PreviousPaidPercentageOfRTR       0.409836
LastBankStatement_OverdraftDays           0.409836
NewDeal_YN                                0.000000
FundedWriteOffAmount_Gross                0.000000
Paybackamount                             0.000000
ActiveDeal_YN                             0.000000
FundedAmount                              0.000000
WriteOff_YN                               0.000000
OpportunityID                             0.000000
WOGross_AdvanceBalance         

# Check Anomalies

1. Perform EDA
2. check wos - remove missing but give in excel sheet:
3. Specifically take care of no of bank statements and all other attributes related to it:
4. Check again the pace rates - {check for extreme weird values} - min_val_check
5. Check for anomalies - time in business -ve_values
6. is this business home based - flag based  -
7. take care of features to be dropped - take previous references
8. fill the missing values - so as to keep the progress alive: 
9. EDA is completed - save the file

In [65]:
numerical_attributes = data.columns[data.dtypes!="object"]
categorical_attributes = data.columns[data.dtypes=="object"]
print("Numerical_attributes :", len(numerical_attributes), "\n")
print(numerical_attributes)
print('\n')
print("Categorical_attributes :", len(categorical_attributes), "\n")
print(categorical_attributes)

Numerical_attributes : 41 

Index(['CreatedDate', 'FundedDate', 'Renewal_PreviousPace',
       'Renewal_PreviousPaidPercentageOfRTR', 'Renewal_PreviousCreditScore',
       'Renewal_CountPreviousDeals', 'Renewal_PreviousCountBounces',
       'Renewal_PreviousPosition', 'Renewal_PreviousHBwLCF',
       'Renewal_PreviousHBwoLCF', 'Renewal_PreviousAvgMonthRevenue',
       'Renewal_PreviousAvgDailyBalance', 'Renewal_TwoWeekBounces',
       'NumberOfBankStatements', 'BankStatement_MA_OverdraftDays',
       'BankStatement_MA_NumReturnItems', 'BankStatement_Sum_NumReturnItems',
       'LastBankStatement_OverdraftDays', 'Credit_Score__c',
       'Time_in_Business__c', 'BBC_Position__c', 'HB_with_LCF_Payment__c',
       'HBwoLCF', 'Contract_HoldBack__c', 'AvgMonthlydeposits',
       'AvgDailybalance', 'FundedAmount', 'Paybackamount', 'NewDeal_YN',
       'ActiveDeal_YN', 'WriteOff_YN', 'FundedWriteOffAmount_Gross',
       'WOGross_AdvanceBalance', 'NetWO', 'Avg_ReturnItemdays',
       'LatestMon

# Dropped the unwanted features

In [66]:
# Dropped unwanted features
data =  data.drop(['OpportunityName',
                   'Priordeal_OppID',
                   'CreatedDate',
                   'FundedDate', 
                   'Industry_Group__c',
                   'Renewal_PreviousCollectionStatus',
                   'ActiveDeal_YN',
                   'NewDeal_YN',
                   'Renewal_PreviousCreditScore',
                   'BankStatement_Sum_NumReturnItems',
                   'LastBankStatement_OverdraftDays',
                   'HBwoLCF',
                   'Contract_HoldBack__c',
                   'Renewal_PreviousHBwoLCF',
                   'NetWO'], axis=1)

In [67]:
data.shape

(488, 34)

In [68]:
numerical_attributes = data.columns[data.dtypes!="object"]
categorical_attributes = data.columns[data.dtypes=="object"]
print("Numerical_attributes :", len(numerical_attributes), "\n")
print(numerical_attributes)
print('\n')
print("Categorical_attributes :", len(categorical_attributes), "\n")
print(categorical_attributes)

Numerical_attributes : 30 

Index(['Renewal_PreviousPace', 'Renewal_PreviousPaidPercentageOfRTR',
       'Renewal_CountPreviousDeals', 'Renewal_PreviousCountBounces',
       'Renewal_PreviousPosition', 'Renewal_PreviousHBwLCF',
       'Renewal_PreviousAvgMonthRevenue', 'Renewal_PreviousAvgDailyBalance',
       'Renewal_TwoWeekBounces', 'NumberOfBankStatements',
       'BankStatement_MA_OverdraftDays', 'BankStatement_MA_NumReturnItems',
       'Credit_Score__c', 'Time_in_Business__c', 'BBC_Position__c',
       'HB_with_LCF_Payment__c', 'AvgMonthlydeposits', 'AvgDailybalance',
       'FundedAmount', 'Paybackamount', 'WriteOff_YN',
       'FundedWriteOffAmount_Gross', 'WOGross_AdvanceBalance',
       'Avg_ReturnItemdays', 'LatestMonth_ReturnItemDays',
       'PctChangeInReveune', 'PctAvgDailybalance2AvgMonthlyDeposits',
       'TwoMonthAvg_AdjustedMonthlyDeposits', 'AdjustedMonthlyDeposits',
       'drop_percent'],
      dtype='object')


Categorical_attributes : 4 

Index(['OpportunityID

###  Observations

* Some attributes name updated, like -
   - FundedWriteOffAmount_Net = NetWO
   - AvgMonthRevenue = AvgMonthlydeposits                        
   - AvgDailyBalance = AvgDailybalance
   - Avg_ReturnItemsDays = Avg_ReturnItemdays                        
   - Pct change in Month Revenue = PctChangeInReveune
   - PctAvgDailybalance2AvgMonthlyRevenue = PctAvgDailybalance2AvgMonthlyDeposits     


* check - WriteOff_YN = should not be any missing values 
* Is_this_Business_home_based__c = Categorical to Numerical 
* Renewal_PreviousPace = remove anomoloies < 50% values 
* Time_in_Business__c = check and drop negative values  

### New added columns for decile analysis
* Paybackamount
* WOGross_AdvanceBalance

In [69]:
# check %_of_missing_values
data.isnull().sum().sort_values(ascending = False)/len(data)*100

LatestMonth_ReturnItemDays               34.016393
PctChangeInReveune                        2.254098
Credit_Score__c                           1.844262
Renewal_PreviousAvgMonthRevenue           1.639344
Renewal_PreviousAvgDailyBalance           1.639344
Renewal_PreviousPaidPercentageOfRTR       0.409836
Renewal_PreviousPace                      0.409836
Industry                                  0.000000
WriteOff_YN                               0.000000
FundedWriteOffAmount_Gross                0.000000
WOGross_AdvanceBalance                    0.000000
OpportunityID                             0.000000
Avg_ReturnItemdays                        0.000000
FundedAmount                              0.000000
PctAvgDailybalance2AvgMonthlyDeposits     0.000000
TwoMonthAvg_AdjustedMonthlyDeposits       0.000000
AdjustedMonthlyDeposits                   0.000000
drop_percent                              0.000000
Paybackamount                             0.000000
HB_with_LCF_Payment__c         

In [70]:
# with pd.ExcelWriter('/content/drive/MyDrive/missing_values_v7_train_data.xlsx') as writer:
#   data[data['PctChangeInReveune'].isna()].to_excel(writer, sheet_name = 'PctChangeInReveune')
#   data[data['LatestMonth_ReturnItemDays'].isna()].to_excel(writer, sheet_name = 'LatestMonth_ReturnItemDays')
#   data[data['Renewal_PreviousPaidPercentageOfRTR'].isna()].to_excel(writer, sheet_name = 'Renewal_PreviousPaidPercentageOfRTR')
#   data[data['Renewal_PreviousAvgMonthRevenue'].isna()].to_excel(writer, sheet_name = 'Renewal_PreviousAvgMonthRevenue')
#   data[data['PctAvgDailybalance2AvgMonthlyDeposits'].isna()].to_excel(writer, sheet_name = 'PctAvgDailybalance2AvgMonthlyDeposits')
#   data[data['AvgMonthlydeposits'].isna()].to_excel(writer, sheet_name = 'AvgMonthlydeposits')
#   data[data['Renewal_PreviousAvgDailyBalance'].isna()].to_excel(writer, sheet_name = 'Renewal_PreviousAvgDailyBalance')
#   data[data['AvgDailybalance'].isna()].to_excel(writer, sheet_name = 'AvgDailybalance')
#   data[data['Avg_ReturnItemdays'].isna()].to_excel(writer, sheet_name = 'Avg_ReturnItemdays')
#   data[data['BankStatement_MA_OverdraftDays'].isna()].to_excel(writer, sheet_name = 'BankStatement_MA_OverdraftDays')
#   data[data['BankStatement_MA_NumReturnItems'].isna()].to_excel(writer, sheet_name = 'BankStatement_MA_NumReturnItems')
#   data[data['Credit_Score__c'].isna()].to_excel(writer, sheet_name = 'Credit_Score__c')
#   data[data['Renewal_PreviousPace'].isna()].to_excel(writer, sheet_name = 'Renewal_PreviousPace')

In [71]:
data.corr()['WriteOff_YN'].sort_values()

NumberOfBankStatements                  -0.168868
Renewal_CountPreviousDeals              -0.108062
AvgDailybalance                         -0.104043
Renewal_PreviousAvgDailyBalance         -0.080234
Renewal_PreviousPaidPercentageOfRTR     -0.078289
BankStatement_MA_OverdraftDays          -0.069259
Paybackamount                           -0.068494
FundedAmount                            -0.068365
Time_in_Business__c                     -0.064839
TwoMonthAvg_AdjustedMonthlyDeposits     -0.055974
PctAvgDailybalance2AvgMonthlyDeposits   -0.054628
Avg_ReturnItemdays                      -0.044075
Credit_Score__c                         -0.039941
Renewal_PreviousPace                    -0.038024
Renewal_PreviousAvgMonthRevenue         -0.035984
BankStatement_MA_NumReturnItems         -0.030310
HB_with_LCF_Payment__c                  -0.027820
AvgMonthlydeposits                      -0.014080
LatestMonth_ReturnItemDays              -0.010210
Renewal_PreviousHBwLCF                  -0.008105


# Tasks to do -

* EDA - 
* -ves, replace it by 0    - WOGross_AdvanceBalance - **Done**
* 1 outlier - remove it - **Done**
* Pace rate - drop it <50%   - Renewal_PreviousPace - **Done**
* check for sir answer - **Done**
* decile analysis on and train data both 
* Contact us - to discuss hyper-parameter tuning

# WriteOff_YN

In [72]:
data['WriteOff_YN'].value_counts()

0    373
1    115
Name: WriteOff_YN, dtype: int64

In [73]:
data['WriteOff_YN'].isna().sum()

0

In [74]:
# data[data['WriteOff_YN'].isna()]['WriteOff_YN']

In [75]:
# data.drop(index=[3138, 3139], inplace = True)

In [76]:
data['WriteOff_YN'].isna().sum()

0

In [77]:
# check for %_skewness
print("Percenatge of Non_writeoff_deal: ", round(data['WriteOff_YN'].value_counts()[0]/len(data['WriteOff_YN'])*100), "%")
print("Percenatge of writeoff_deal: ", round(data['WriteOff_YN'].value_counts()[1]/len(data['WriteOff_YN'])*100), "%")

Percenatge of Non_writeoff_deal:  76 %
Percenatge of writeoff_deal:  24 %


In [78]:
## Dealing with the categorical attributes - converting them with one hot encoding methods

#### Is_this_Business_home_based__c
#### Industry
#### Drop_category 

In [79]:
data = pd.get_dummies(data, columns = ['Is_this_Business_home_based__c'],drop_first= True)
data = pd.get_dummies(data, columns = ["Industry"], drop_first = True)
data = pd.get_dummies(data, columns = ["Drop_category"], drop_first = True)

In [80]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 488 entries, 0 to 487
Data columns (total 41 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   OpportunityID                               488 non-null    object 
 1   Renewal_PreviousPace                        486 non-null    float64
 2   Renewal_PreviousPaidPercentageOfRTR         486 non-null    float64
 3   Renewal_CountPreviousDeals                  488 non-null    int64  
 4   Renewal_PreviousCountBounces                488 non-null    int64  
 5   Renewal_PreviousPosition                    488 non-null    int64  
 6   Renewal_PreviousHBwLCF                      488 non-null    float64
 7   Renewal_PreviousAvgMonthRevenue             480 non-null    float64
 8   Renewal_PreviousAvgDailyBalance             480 non-null    float64
 9   Renewal_TwoWeekBounces                      488 non-null    int64  
 10  NumberOfBankSt

# Renewal_PreviousPace

In [81]:
data[data['Renewal_PreviousPace'].isna()]['Renewal_PreviousPace'] #fill this up

49    NaN
212   NaN
Name: Renewal_PreviousPace, dtype: float64

In [82]:
data[data['Renewal_PreviousPace'] < 30][['Renewal_PreviousPace',"WriteOff_YN", "OpportunityID"]]

,Renewal_PreviousPace,WriteOff_YN,OpportunityID
110,10.04,0,0062M00000o9rEEQAY
223,23.22,1,0062M00000oBctkQAC


In [83]:
data.drop(index=[110,223], inplace = True)

In [84]:
data[data['Renewal_PreviousPace'] < 30]['Renewal_PreviousPace']

Series([], Name: Renewal_PreviousPace, dtype: float64)

In [85]:
# data[data['Renewal_PreviousPace'] < 50][['Renewal_PreviousPace', 'OpportunityID']].to_excel('/content/drive/MyDrive/anomalies_pace_rate.xlsx', index = True)

#### Time_in_Business__c - cheking -ve values

In [86]:
data[data["Time_in_Business__c"]<0]["Time_in_Business__c"]

Series([], Name: Time_in_Business__c, dtype: float64)

In [87]:
#data["Time_in_Business__c"] = data["Time_in_Business__c"].replace([-0.52,-0.85, -0.12, -1.07, -0.91, -0.72, -0.08, -0.06, -0.65, -0.37, -0.24], data["Time_in_Business__c"].median())

#### WOGross_AdvanceBalance


1.   Look for -ve values and replace them with zero

In [88]:
data['WOGross_AdvanceBalance'].describe()

count       486.000000
mean       3948.086461
std       18147.717532
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max      344226.190000
Name: WOGross_AdvanceBalance, dtype: float64

In [89]:
data[data['WOGross_AdvanceBalance'] > 0]['WOGross_AdvanceBalance']

5      344226.19
7       30839.75
8        6672.00
10      14354.45
13      12091.14
         ...    
465     65577.24
472     17164.73
479      2881.08
480     91386.70
482     14023.55
Name: WOGross_AdvanceBalance, Length: 114, dtype: float64

In [90]:
data[(data['WOGross_AdvanceBalance'] < 0) & data['WriteOff_YN'] == 1][['WOGross_AdvanceBalance', 'WriteOff_YN']][:10]

,WOGross_AdvanceBalance,WriteOff_YN


In [91]:
# Thesre are technically the anomalies - need to treat them differently at the decile time 
data[(data['WOGross_AdvanceBalance'] == 0) & data['WriteOff_YN'] == 1][['WOGross_AdvanceBalance', 'FundedWriteOffAmount_Gross', 'WriteOff_YN']]

,WOGross_AdvanceBalance,FundedWriteOffAmount_Gross,WriteOff_YN


In [92]:
len(data[(data['WOGross_AdvanceBalance'] == 0) & data['WriteOff_YN'] == 1][['WOGross_AdvanceBalance', 'WriteOff_YN']])

0

In [93]:
data[data['WOGross_AdvanceBalance'] < 0]['WOGross_AdvanceBalance']

Series([], Name: WOGross_AdvanceBalance, dtype: float64)

In [94]:
data[data['WOGross_AdvanceBalance'] < 0]['WriteOff_YN'].value_counts()

Series([], Name: WriteOff_YN, dtype: int64)

In [95]:
data[(data['WOGross_AdvanceBalance'] > 0) & data['WriteOff_YN'] == 1][['WOGross_AdvanceBalance', 'WriteOff_YN']]

,WOGross_AdvanceBalance,WriteOff_YN
5,344226.19,1
7,30839.75,1
8,6672.00,1
10,14354.45,1
13,12091.14,1
...,...,...
465,65577.24,1
472,17164.73,1
479,2881.08,1
480,91386.70,1


In [96]:
data[data["WOGross_AdvanceBalance"] > 0]["WriteOff_YN"].value_counts()

1    114
Name: WriteOff_YN, dtype: int64

In [97]:
# replacing -ve values with zeros
# data['WOGross_AdvanceBalance'] = data['WOGross_AdvanceBalance'].apply(lambda x: 0 if x < 0 else x)

In [98]:
data[data['WOGross_AdvanceBalance'] == 0]['WriteOff_YN'].value_counts()

0    372
Name: WriteOff_YN, dtype: int64

In [99]:
'''
funded_amount = $1000
payback_amount = $1500
collected_amount = $800 || $1200 || $1500 (extra $200)
adv_wos_$ (opp_1) = $1500 - $800 = $700
adv_wos_$ (opp_1) = $1500 - $1200 = $300
adv_wos_$ (opp_1) = $1500 - ($1500 + $200) = -$200
'''


'\nfunded_amount = $1000\npayback_amount = $1500\ncollected_amount = $800 || $1200 || $1500 (extra $200)\nadv_wos_$ (opp_1) = $1500 - $800 = $700\nadv_wos_$ (opp_1) = $1500 - $1200 = $300\nadv_wos_$ (opp_1) = $1500 - ($1500 + $200) = -$200\n'

In [100]:
data[(data['WOGross_AdvanceBalance'] == 0) & (data['WriteOff_YN'] == 1)][["OpportunityID", "WOGross_AdvanceBalance", "WriteOff_YN"]]

,OpportunityID,WOGross_AdvanceBalance,WriteOff_YN


In [ ]:
#data[(data['WOGross_AdvanceBalance'] == 0) & (data['WriteOff_YN'] == 1)][["OpportunityID", "WOGross_AdvanceBalance", "WriteOff_YN"]].index

In [ ]:
'''
data.drop(index= [38,  163,  208,  274,  803,  814,  822,  892,  949, 1310, 1422,
            1493, 1503, 1561, 1675, 1745, 2242, 2353, 2356, 2378, 2408, 2436,
            2481, 2490, 2534, 2639, 2682, 2706, 2740, 2779, 2911, 2986, 3135], inplace = True)
'''

#### BBC_Position__C

In [101]:
data['BBC_Position__c']

0      1
1      3
2      1
3      2
4      1
      ..
483    1
484    1
485    1
486    2
487    2
Name: BBC_Position__c, Length: 486, dtype: int64

In [102]:
data['BBC_Position__c'].value_counts()

1    247
2    119
3     68
4     32
5     16
6      3
7      1
Name: BBC_Position__c, dtype: int64

In [103]:
data['BBC_Position__c'].describe()

count    486.000000
mean       1.897119
std        1.161989
min        1.000000
25%        1.000000
50%        1.000000
75%        2.000000
max        7.000000
Name: BBC_Position__c, dtype: float64

In [104]:
data['BBC_Position__c'].isna().sum()

0

In [105]:
data[data['BBC_Position__c'] < 0]['BBC_Position__c']


Series([], Name: BBC_Position__c, dtype: int64)

# NumberOfBankStatements                    

In [ ]:
# Related attributes -

# AvgMonthRevenue - column name updated
# Renewal_PreviousAvgMonthRevenue
# AvgDailyBalance - column name updated
# Renewal_PreviousAvgDailyBalance
# BankStatement_MA_OverdraftDays
# BankStatement_MA_NumReturnItems
# Avg_ReturnItemsDays - column name updated
# Pct change in Month Revenue  - column name updated
# PctAvgDailybalance2AvgMonthlyRevenue - column name updated
# LastBankStatement_OverdraftDays - dropped
# BankStatement_Sum_NumReturnItems - dropped


In [106]:
data.columns

Index(['OpportunityID', 'Renewal_PreviousPace',
       'Renewal_PreviousPaidPercentageOfRTR', 'Renewal_CountPreviousDeals',
       'Renewal_PreviousCountBounces', 'Renewal_PreviousPosition',
       'Renewal_PreviousHBwLCF', 'Renewal_PreviousAvgMonthRevenue',
       'Renewal_PreviousAvgDailyBalance', 'Renewal_TwoWeekBounces',
       'NumberOfBankStatements', 'BankStatement_MA_OverdraftDays',
       'BankStatement_MA_NumReturnItems', 'Credit_Score__c',
       'Time_in_Business__c', 'BBC_Position__c', 'HB_with_LCF_Payment__c',
       'AvgMonthlydeposits', 'AvgDailybalance', 'FundedAmount',
       'Paybackamount', 'WriteOff_YN', 'FundedWriteOffAmount_Gross',
       'WOGross_AdvanceBalance', 'Avg_ReturnItemdays',
       'LatestMonth_ReturnItemDays', 'PctChangeInReveune',
       'PctAvgDailybalance2AvgMonthlyDeposits',
       'TwoMonthAvg_AdjustedMonthlyDeposits', 'AdjustedMonthlyDeposits',
       'drop_percent', 'Is_this_Business_home_based__c_Yes',
       'Industry_Construction', 'Industry

In [107]:
print('Checking Missing Values -')
print('\n')
print('NumberOfBankStatements :', data['NumberOfBankStatements'].isna().sum())
print('AvgMonthlydeposits :' ,data['AvgMonthlydeposits'].isna().sum())          
print('AvgDailybalance :' ,data['AvgDailybalance'].isna().sum())
print('BankStatement_MA_NumReturnItems :', data['BankStatement_MA_NumReturnItems'].isna().sum())
print('BankStatement_MA_OverdraftDays :' ,data['BankStatement_MA_OverdraftDays'].isna().sum())

Checking Missing Values -


NumberOfBankStatements : 0
AvgMonthlydeposits : 0
AvgDailybalance : 0
BankStatement_MA_NumReturnItems : 0
BankStatement_MA_OverdraftDays : 0


In [108]:
def attribute_info(df_feature):
  print('Description of attribute :', '\n' )
  print(df_feature.describe(), '\n')
  print('Value counts of attribute :', '\n' )
  print(df_feature.value_counts(), '\n')
  print('Unique number of attribute :', '\n' )
  print(df_feature.unique(), '\n')
  print('Missing values in attribute :', '\n' )
  print(df_feature.isna().sum(), '\n')
  print('Missing values in attribute_% :', '\n' )
  print(df_feature.isna().sum()/len(df_feature)*100, '\n')
  plt.figure(figsize=(20,10))
  plt.subplot(2,2,1)
  sns.distplot(df_feature)
  plt.subplot(2,2,2)
  sns.boxplot(df_feature)
  plt.show()

In [109]:
data.isnull().sum().sort_values(ascending = False)/len(data)*100

LatestMonth_ReturnItemDays                    33.950617
PctChangeInReveune                             2.057613
Credit_Score__c                                1.851852
Renewal_PreviousAvgMonthRevenue                1.440329
Renewal_PreviousAvgDailyBalance                1.440329
Renewal_PreviousPaidPercentageOfRTR            0.411523
Renewal_PreviousPace                           0.411523
OpportunityID                                  0.000000
PctAvgDailybalance2AvgMonthlyDeposits          0.000000
TwoMonthAvg_AdjustedMonthlyDeposits            0.000000
AdjustedMonthlyDeposits                        0.000000
drop_percent                                   0.000000
Is_this_Business_home_based__c_Yes             0.000000
Industry_Finance, Insurance, Real Estate       0.000000
Industry_Construction                          0.000000
WOGross_AdvanceBalance                         0.000000
Industry_Manufacturing                         0.000000
Industry_Mining                                0

### Mean and Median values of training set features - 

In [110]:
# BankStatement_MA_OverdraftDays :                            0.1
# AvgMonthlydeposits :                                        28061.5156665
# BankStatement_MA_NumReturnItems :                           0.0
# AvgDailybalance :                                           3557.125
# PctAvgDailybalance2AvgMonthlyDeposits :                     0.1223568549836635
# Avg_ReturnItemdays :                                        0.0
# drop_percent :                                              -1.183065
# TwoMonthAvg_AdjustedMonthlyDeposits :                       28870.52
# AdjustedMonthlyDeposits :                                   29434.97

## Need to impute these values in test set - 

In [115]:
# LatestMonth_ReturnItemDays :                                0.0
# PctChangeInReveune :                                        0.051044570169082995
# Credit_Score__c :                                           585.9409153005464       ## Mean
# Renewal_PreviousAvgMonthRevenue :                           25553.69
# Renewal_PreviousAvgDailyBalance :                           3535.52
# Renewal_PreviousPaidPercentageOfRTR :                       0.67
# Renewal_PreviousPace :                                      100.0


In [112]:
data[data['PctChangeInReveune'].isna()]['PctChangeInReveune']

12    NaN
19    NaN
28    NaN
34    NaN
152   NaN
180   NaN
205   NaN
214   NaN
226   NaN
428   NaN
Name: PctChangeInReveune, dtype: float64

In [114]:
data[data['Credit_Score__c'].isna()]['Credit_Score__c']

28    NaN
185   NaN
238   NaN
295   NaN
297   NaN
335   NaN
428   NaN
450   NaN
482   NaN
Name: Credit_Score__c, dtype: float64

In [116]:
data['LatestMonth_ReturnItemDays'].fillna(0.0, inplace = True)
data['PctChangeInReveune'].fillna(0.051044570169082995, inplace = True)
data['Credit_Score__c'].fillna(585.9409153005464, inplace = True)               # Imputed by Credit_score Mean
data['Renewal_PreviousAvgMonthRevenue'].fillna(25553.69, inplace = True)
data['Renewal_PreviousAvgDailyBalance'].fillna(3535.52, inplace = True)
data['Renewal_PreviousPaidPercentageOfRTR'].fillna(0.67, inplace = True)
data['Renewal_PreviousPace'].fillna(100.0, inplace = True)


In [119]:
print(data['PctChangeInReveune'][152])
print(data['PctChangeInReveune'][12])

0.051044570169082995
0.051044570169082995


In [120]:
print(data['Credit_Score__c'][28])
print(data['Credit_Score__c'][185])

585.9409153005464
585.9409153005464


In [121]:
data.isnull().sum().sort_values(ascending = False)/len(data)*100

OpportunityID                                 0.0
WriteOff_YN                                   0.0
WOGross_AdvanceBalance                        0.0
Avg_ReturnItemdays                            0.0
LatestMonth_ReturnItemDays                    0.0
PctChangeInReveune                            0.0
PctAvgDailybalance2AvgMonthlyDeposits         0.0
TwoMonthAvg_AdjustedMonthlyDeposits           0.0
AdjustedMonthlyDeposits                       0.0
drop_percent                                  0.0
Is_this_Business_home_based__c_Yes            0.0
Industry_Construction                         0.0
Industry_Finance, Insurance, Real Estate      0.0
Industry_Manufacturing                        0.0
Industry_Mining                               0.0
Industry_Retail                               0.0
Industry_Services                             0.0
Industry_Transportation & Public Utilities    0.0
Industry_Wholesale                            0.0
FundedWriteOffAmount_Gross                    0.0


In [123]:
data.shape

(486, 41)

In [124]:
data.to_excel('/content/drive/MyDrive/LCF_Internal/Renewable_Model/EDA Approach-1/XGBoost Implementation/logistic_v9/v9_after_eda_test.xlsx', index = False)

In [49]:
# missing_attributes = ['LatestMonth_ReturnItemDays', 
#                       'PctChangeInReveune', 
#                       'Renewal_PreviousAvgMonthRevenue',
#                       'Renewal_PreviousAvgDailyBalance',
#                       'PctAvgDailybalance2AvgMonthlyDeposits',
#                       'Avg_ReturnItemdays',
#                       'Renewal_PreviousPaidPercentageOfRTR', 
#                       'BankStatement_MA_OverdraftDays',           
#                       'AvgMonthlydeposits',        
#                       'BankStatement_MA_NumReturnItems',          
#                       'AvgDailybalance',                                            
#                       'Credit_Score__c',                          
#                       'Renewal_PreviousPace',
#                       'drop_percent',
#                       'TwoMonthAvg_AdjustedMonthlyDeposits',
#                       'AdjustedMonthlyDeposits']    

# for col in missing_attributes:
#   print('\033[1m'+'attribute_name :', col+'\033[0m')
#   attribute_info(data[col])

In [ ]:
# data['Renewal_PreviousPaidPercentageOfRTR'].fillna(data['Renewal_PreviousPaidPercentageOfRTR'].median(), inplace = True)
# data['BankStatement_MA_OverdraftDays'].fillna(data['BankStatement_MA_OverdraftDays'].median(), inplace = True)
# data['AvgMonthlydeposits'].fillna(data['AvgMonthlydeposits'].median(), inplace = True)
# data['BankStatement_MA_NumReturnItems'].fillna(data['BankStatement_MA_NumReturnItems'].median(), inplace = True)
# data['AvgDailybalance'].fillna(data['AvgDailybalance'].median(), inplace = True)
# data['LatestMonth_ReturnItemDays'].fillna(data['LatestMonth_ReturnItemDays'].median(),inplace = True)
# data['PctChangeInReveune'].fillna(data['PctChangeInReveune'].median(),inplace = True)
# data['PctAvgDailybalance2AvgMonthlyDeposits'].fillna(data['PctAvgDailybalance2AvgMonthlyDeposits'].median(),inplace = True)
# data['Avg_ReturnItemdays'].fillna(data['Avg_ReturnItemdays'].median(),inplace = True)
# data['Renewal_PreviousAvgMonthRevenue'].fillna(data['Renewal_PreviousAvgMonthRevenue'].median(),inplace = True)
# data['Renewal_PreviousAvgDailyBalance'].fillna(data['Renewal_PreviousAvgDailyBalance'].median(),inplace = True)
# data['Renewal_PreviousPace'].fillna(data['Renewal_PreviousPace'].median(),inplace = True)
# data['drop_percent'].fillna(data['drop_percent'].median(),inplace = True)
# data['TwoMonthAvg_AdjustedMonthlyDeposits'].fillna(data['TwoMonthAvg_AdjustedMonthlyDeposits'].median(),inplace = True)
# data['AdjustedMonthlyDeposits'].fillna(data['AdjustedMonthlyDeposits'].median(),inplace = True)


In [50]:
# credit_mean = data[(data['Credit_Score__c'] != 0) & (data['Credit_Score__c'] != 1)]['Credit_Score__c'].mean()
# credit_mean

In [51]:
# data['Credit_Score__c'].fillna(credit_mean,inplace = True)

In [52]:
# attribute_info(data['Credit_Score__c'])